# 📌 Contexte et Objectifs

Ce rapport présente l'analyse de l'activité des taxis à New York pour le mois sélectionné.

Les objectifs sont les suivants :

* Suivre les indicateurs clés de performance (KPI) des courses : nombre de trajets, distances parcourues et durées.
* Identifier les périodes de forte activité.
* Analyser les tendances et variations dans l'activité des taxis.
* Fournir des visualisations claires pour faciliter la compréhension des données.

# 📥 Préparation et Traitement des Données

Les données taxi sont chargées pour analyse.


In [ ]:
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import os
import re
import yaml
from scipy import stats
import numpy as np
import geopandas as gpd

# Lecture du fichier settings.yaml pour obtenir le mois du rapport
# Si le fichier n'est pas trouvé, utilisez une valeur par défaut
try:
    with open("config/settings.yaml", 'r') as file:
        settings = yaml.safe_load(file)
        month_code = settings.get('report_month', '2023-07')
except Exception as e:
    print(f"⚠️ Erreur lors de la lecture du fichier settings.yaml: {e}")
    month_code = '2023-07'  # Valeur par défaut


# Conversion en format lisible
try:
    date_obj = datetime.strptime(month_code, "%Y-%m")
    month_name = date_obj.strftime("%B %Y")
except:
    month_name = "Inconnu"


# Chargement des données taxi uniquement
# 📦 Chargement et nettoyage des données taxi
# Chargement des données taxi uniquement
try:
    df_taxi = pd.read_csv(f"data/{month_code}-taxi.csv", parse_dates=["pickup_datetime", "dropoff_datetime"])

    # Préparation des données
    df_taxi["hour"] = df_taxi["pickup_datetime"].dt.hour
    df_taxi["weekday"] = df_taxi["pickup_datetime"].dt.day_name()
    df_taxi["date"] = df_taxi["pickup_datetime"].dt.date
    df_taxi["duration"] = (df_taxi["dropoff_datetime"] - df_taxi["pickup_datetime"]).dt.total_seconds() / 60

    # Filtrer les durées négatives ou aberrantes (>180 min)
    df_taxi = df_taxi[(df_taxi["duration"] > 0) & (df_taxi["duration"] < 180)]

    # Filtrer les distances aberrantes (>50 miles)
    df_taxi = df_taxi[df_taxi["trip_distance"] < 50]

    # Utilisation directe des données taxi pour l'analyse
    df = df_taxi.copy()
except Exception as e:
    print(f"❌ Erreur lors du chargement des données taxi: {e}")
    # Création d'un DataFrame vide pour éviter les erreurs
    df = pd.DataFrame(columns=[
        "pickup_datetime",
        "dropoff_datetime",
        "trip_distance",
        "passenger_count",
        "payment_type",
        "total_amount",
        "PULocationID",
        "DOLocationID",
        "tip_amount",
        "hour",
        "weekday",
        "date",
        "duration",
        "payment_method"
    ])

    print("⚠️ Aucune donnée taxi disponible, rapport limité")


In [ ]:
#| output: asis
# Génération "à la volée" du bloc d'information sur la période en Markdown
print(f"## Configuration pour le rapport du mois de **{month_name}**")
print()
print(f"- 📅 Période d'analyse : **{month_name}**")
print(f"- 🗓️ Code période : **{month_code}**")
print(f"- 📊 Fichier de données : **data/{month_code}-taxi.csv**")
print(f"- 🔢 Nombre d'enregistrements : **{len(df):,}**")
# 📈 Indicateurs Clés de Performance (KPIs)

In [ ]:
#| echo: false

# Calcul des KPIs avec gestion des erreurs
if not df.empty:
    total_trips = len(df)
    avg_distance = df["trip_distance"].mean() if "trip_distance" in df.columns else 0
    avg_distance_rounded = round(avg_distance, 2)

    if "duration" in df.columns:
        avg_duration = df["duration"].mean()
        # Identification des heures de pointe basée sur les données réelles
        hourly_trips = df.groupby("hour").size()
        peak_hours = hourly_trips.sort_values(ascending=False).head(3).index.tolist()
        peak_hours.sort()  # Trier les heures par ordre croissant
        peak_hours_str = ", ".join([f"{h}h" for h in peak_hours])
    else:
        avg_duration = 0
        peak_hours_str = "Non déterminé"

    # Jour de la semaine le plus chargé
    if "weekday" in df.columns:
        busiest_day = df.groupby("weekday").size().idxmax()
    else:
        busiest_day = "Non déterminé"
else:
    total_trips = 0
    avg_distance = 0
    avg_distance_rounded = 0
    avg_duration = 0
    peak_hours_str = "Non déterminé"
    busiest_day = "Non déterminé"

In [ ]:
#| output: asis
# Génération "à la volée" du bloc KPI en Markdown
print(f"## Rapport de trafic du mois de **{month_name}**")
print()
print(f"- 📊 Nombre total de trajets : **{total_trips:,}**")
print(f"- 🛣️ Distance moyenne parcourue : **{avg_distance_rounded:.1f}** miles")
print(f"- ⏱️ Durée moyenne d’un trajet : **{avg_duration:.1f}** minutes")
print(f"- 🕘 Heures de pointe : **{peak_hours_str}**")
print(f"- 📅 Jour le plus chargé : **{busiest_day}**")

# 📊 Analyse de l'Activité par Heure de la Journée

## Volume de trajets par heure

Ce graphique présente la répartition du nombre de trajets en fonction de l'heure de la journée.


In [ ]:
#| fig-cap: " "

# Vérification que df existe et contient la colonne "hour"
if 'df' in globals() and not df.empty and "hour" in df.columns:
    # Préparation des données
    hourly_counts = df["hour"].value_counts().sort_index().reset_index()
    hourly_counts.columns = ["hour", "count"]

    # Heure avec le plus de trajets
    max_hour = hourly_counts.loc[hourly_counts["count"].idxmax()]

    # Création du graphique avec Plotly
    fig = px.bar(hourly_counts, x="hour", y="count",
                 labels={"hour": "Heure", "count": "Nombre de trajets"},
                 title=" ")

    # Mise en forme du graphique
    fig.update_layout(
        xaxis=dict(
            tickmode='linear',
            tick0=0,
            dtick=1,
            title=" "
        ),
        yaxis=dict(
            title=" "
        ),
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        height=600,
        width=1200
    )

    # Ajout d'annotation pour le pic
    fig.add_annotation(
        x=max_hour["hour"],
        y=max_hour["count"],
        text=f"Pic: {max_hour['count']:,} trajets",
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40
    )

    fig.show()

    # Stockage de l'information pour l'affichage
    peak_hour = int(max_hour["hour"])

    # Détermination du message approprié
    if peak_hour < 6:
        observation_msg = f"Le pic d'activité est observé très tôt, à **{peak_hour}h**, suggérant une forte demande matinale."
    elif 6 <= peak_hour < 10:
        observation_msg = f"Le pic d'activité est observé à **{peak_hour}h**, correspondant à l'heure de pointe matinale."
    elif 10 <= peak_hour < 16:
        observation_msg = f"Le pic d'activité est observé à **{peak_hour}h**, durant la période de mi-journée."
    elif 16 <= peak_hour < 20:
        observation_msg = f"Le pic d'activité est observé à **{peak_hour}h**, correspondant à l'heure de pointe en fin de journée."
    else:
        observation_msg = f"Le pic d'activité est observé en soirée, à **{peak_hour}h**, suggérant une forte demande nocturne."
else:
    print("⚠️ Données insuffisantes pour générer le graphique des trajets par heure")
    observation_msg = "Données insuffisantes pour générer l'observation."

In [ ]:
#| output: asis

# Utilisation de la même technique avec output: asis pour générer le bloc d'observation
if 'observation_msg' in globals():
    print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
    print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
    print(f'  <p>{observation_msg}</p>')
    print('</div>')

## Distance moyenne parcourue par heure


In [ ]:
#| fig-cap: " "

if not df.empty and "hour" in df.columns and "trip_distance" in df.columns:
    # Calcul des distances moyennes par heure
    dist_hour = df.groupby("hour")["trip_distance"].mean().reset_index()

    # Identification de l'heure avec la distance moyenne maximale et minimale
    max_dist_hour = dist_hour.loc[dist_hour["trip_distance"].idxmax()]
    min_dist_hour = dist_hour.loc[dist_hour["trip_distance"].idxmin()]

    # Création du graphique avec Plotly
    fig = go.Figure()

    # Ajout de la ligne principale
    fig.add_trace(go.Scatter(
        x=dist_hour["hour"],
        y=dist_hour["trip_distance"],
        mode='lines+markers',
        name='Distance moyenne',
        line=dict(color='royalblue', width=3),
        marker=dict(size=8)
    ))

    # Mise en forme du graphique
    fig.update_layout(
        title="Distance moyenne par heure",
        xaxis=dict(
            title=" ",
            tickmode='linear',
            tick0=0,
            dtick=1
        ),
        yaxis=dict(
            title="Distance moyenne (miles)"
        ),
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        height=600,
        width=1200
    )

    # Ajout d'annotations pour les points maximum et minimum
    fig.add_annotation(
        x=max_dist_hour["hour"],
        y=max_dist_hour["trip_distance"],
        text=f'Max: {max_dist_hour["trip_distance"]:.2f} miles',
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=-40
    )

    fig.add_annotation(
        x=min_dist_hour["hour"],
        y=min_dist_hour["trip_distance"],
        text=f'Min: {min_dist_hour["trip_distance"]:.2f} miles',
        showarrow=True,
        arrowhead=1,
        ax=0,
        ay=40
    )

    fig.show()

    # Stockage de l'information pour l'affichage
    max_hour_dist = int(max_dist_hour["hour"])
    min_hour_dist = int(min_dist_hour["hour"])

    # Création du message d'observation
    observation_msg = f"Les trajets les plus longs ({max_dist_hour['trip_distance']:.2f} miles) sont observés à {max_hour_dist}h, "
    observation_msg += f"tandis que les plus courts ({min_dist_hour['trip_distance']:.2f} miles) sont à {min_hour_dist}h."

    # Ajout d'un commentaire contextuel selon l'heure du maximum
    if max_hour_dist >= 22 or max_hour_dist <= 5:
        observation_msg += " Les distances plus longues en période nocturne pourraient s'expliquer par des trajets vers/depuis les aéroports ou la périphérie."
    elif 16 <= max_hour_dist < 20:
        observation_msg += " Les distances plus longues en fin de journée pourraient correspondre à des retours en banlieue après le travail."
    elif 6 <= max_hour_dist < 10:
        observation_msg += " Les distances plus longues tôt le matin pourraient correspondre à des trajets domicile-travail depuis la périphérie."
else:
    print("⚠️ Données insuffisantes pour générer le graphique des distances moyennes par heure")
    observation_msg = "Données insuffisantes pour analyser les distances par heure."

In [ ]:
#| output: asis

# Affichage du bloc d'observation formaté
if 'observation_msg' in globals():
    print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
    print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
    print(f'  <p>{observation_msg}</p>')
    print('</div>')

# 📊 Analyse de la Durée des Trajets


In [ ]:
#| fig-cap: " "

if not df.empty and "duration" in df.columns:
    # Filtrage des durées aberrantes (> 2h ou < 0)
    valid_durations = df[(df["duration"] > 0) & (df["duration"] < 120)]

    # Calcul des statistiques
    mean_duration = valid_durations["duration"].mean()
    median_duration = valid_durations["duration"].median()
    percentile_90 = valid_durations["duration"].quantile(0.9)

    # Création de l'histogramme avec Plotly
    fig = ff.create_distplot(
        [valid_durations["duration"]],
        group_labels=["Durée des trajets"],
        bin_size=1,
        show_rug=False,
        curve_type='normal'
    )

    y_vals = []
    for trace in fig.data:
        if hasattr(trace, 'y') and trace.y is not None:
            y_vals.extend(trace.y)
    y_max = max(y_vals) if y_vals else 0

    # Lignes verticales moyenne et médiane
    fig.add_shape(type="line",
                  x0=mean_duration, y0=0,
                  x1=mean_duration, y1=y_max,
                  line=dict(color="red", width=2, dash="dash"))
    fig.add_shape(type="line",
                  x0=median_duration, y0=0,
                  x1=median_duration, y1=y_max,
                  line=dict(color="green", width=2, dash="dash"))

    # Mise en page et annotations
    fig.update_layout(
        title="Distribution des Durées de Trajet",
        xaxis_title="Durée (minutes)",
        yaxis_title="Densité",
        width=1200, height=600,
        plot_bgcolor='rgba(240,240,240,0.5)',
        showlegend=False,
        annotations=[
            dict(x=mean_duration, y=y_max*0.95,
                 text=f"Moyenne: {mean_duration:.1f} min",
                 showarrow=False, font=dict(color="red")),
            dict(x=median_duration, y=y_max*0.85,
                 text=f"Médiane: {median_duration:.1f} min",
                 showarrow=False, font=dict(color="green"))
        ]
    )
    fig.show()

    # Calcul pour l'observation
    skew = valid_durations["duration"].skew()

    # Construction du message d'observation
    observation_msg = f"90% des trajets durent moins de {percentile_90:.1f} minutes. "

    # Ajout d'une explication sur la distribution
    if skew > 1:
        observation_msg += "La distribution est fortement asymétrique vers la droite, montrant que la majorité des trajets sont courts, mais quelques trajets très longs influencent la moyenne."
    elif skew > 0.5:
        observation_msg += "La distribution présente une asymétrie modérée vers la droite, suggérant que la plupart des trajets sont de courte durée."
    else:
        observation_msg += "La distribution est relativement symétrique, indiquant une répartition équilibrée des durées de trajet."

    # Ajout d'une observation sur l'écart moyenne/médiane si significatif
    if mean_duration > median_duration * 1.2:
        observation_msg += f" L'écart significatif entre moyenne et médiane ({(mean_duration/median_duration - 1)*100:.0f}% de différence) confirme l'influence des trajets longs sur la distribution."
else:
    print("⚠️ Pas assez de données pour analyser les durées")
    observation_msg = "Données insuffisantes pour analyser les durées de trajet."

In [ ]:
#| output: asis

# Affichage du bloc d'observation formaté
if 'observation_msg' in globals():
    print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
    print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
    print(f'  <p>{observation_msg}</p>')
    print('</div>')

# 📊 Analyse par Jour de la Semaine


In [ ]:
#| fig-cap: " "

if not df.empty and "weekday" in df.columns:
    # Ordre des jours de la semaine
    days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    fr_days = {"Monday": "Lundi", "Tuesday": "Mardi", "Wednesday": "Mercredi",
               "Thursday": "Jeudi", "Friday": "Vendredi", "Saturday": "Samedi", "Sunday": "Dimanche"}

    # Conversion en jours français si nécessaire
    if str(df["weekday"].iloc[0]) in fr_days.values():
        days_order = [fr_days[day] for day in days_order]

    # Préparation des données
    weekday_counts = df["weekday"].value_counts().reindex(days_order).reset_index()
    weekday_counts.columns = ["weekday", "count"]

    # Ajout d'une colonne pour identifier les weekends
    weekday_counts["is_weekend"] = weekday_counts["weekday"].isin([days_order[-1], days_order[-2]])

    # Création du graphique avec Plotly
    fig = px.bar(
        weekday_counts,
        x="weekday",
        y="count",
        color="is_weekend",
        color_discrete_map={True: "orange", False: "royalblue"},
        labels={"weekday": "Jour de la semaine", "count": "Nombre de trajets", "is_weekend": "Weekend"},
        title="Nombre de trajets par jour de la semaine"
    )

    # Mise en forme du graphique
    fig.update_layout(
        xaxis=dict(
            categoryorder='array',
            categoryarray=days_order,
            title=" "
        ),
        yaxis=dict(
            title=" "
        ),
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        height=600,
        width=1200,
        showlegend=False
    )

    fig.show()

    # Identification du jour le plus et moins chargé
    max_day_row = weekday_counts.loc[weekday_counts["count"].idxmax()]
    min_day_row = weekday_counts.loc[weekday_counts["count"].idxmin()]
    max_day = max_day_row["weekday"]
    min_day = min_day_row["weekday"]
    max_count = max_day_row["count"]
    min_count = min_day_row["count"]

    diff_pct = ((max_count - min_count) / min_count) * 100

    # Construction du message d'observation
    observation_msg = f"{max_day} est le jour le plus chargé avec {int(max_count):,} trajets, "
    observation_msg += f"tandis que {min_day} est le moins actif avec {int(min_count):,} trajets, "
    observation_msg += f"soit une différence de {diff_pct:.1f}%. "

    # Analyse weekend vs semaine
    weekend_rows = weekday_counts[weekday_counts["is_weekend"]]
    weekday_rows = weekday_counts[~weekday_counts["is_weekend"]]

    weekend_avg = weekend_rows["count"].mean() if not weekend_rows.empty else 0
    weekday_avg = weekday_rows["count"].mean() if not weekday_rows.empty else 0

    if weekday_avg > 0:
        diff_we_pct = ((weekend_avg - weekday_avg) / weekday_avg) * 100

        if diff_we_pct > 10:
            observation_msg += f"Le weekend connaît une hausse significative d'activité (+{diff_we_pct:.1f}%) par rapport à la semaine, "
            observation_msg += "ce qui pourrait s'expliquer par les sorties nocturnes, événements culturels et touristiques."
        elif diff_we_pct < -10:
            observation_msg += f"Le weekend connaît une baisse significative d'activité ({diff_we_pct:.1f}%) par rapport à la semaine, "
            observation_msg += "suggérant que les déplacements professionnels constituent une part importante de l'activité des taxis."
        else:
            observation_msg += "L'activité reste relativement stable entre les jours de semaine et le weekend, "
            observation_msg += "indiquant un équilibre entre les usages professionnels et de loisirs."
else:
    print("⚠️ Données insuffisantes pour générer le graphique par jour de la semaine")
    observation_msg = "Données insuffisantes pour analyser l'activité par jour de la semaine."

In [ ]:
#| output: asis

# Affichage du bloc d'observation formaté
if 'observation_msg' in globals():
    print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
    print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
    print(f'  <p>{observation_msg}</p>')
    print('</div>')

# 📅 Répartition des Trajets par Jour


In [ ]:
#| fig-cap: " "

if not df.empty and "date" in df.columns:
    # Vérification de la présence de valeurs pour chaque jour du mois
    date_range = pd.date_range(df["date"].min(), df["date"].max())
    daily_counts = df.groupby("date").size().reset_index(name="trips")

    # Conversion au format datetime pour manipulation
    daily_counts["date"] = pd.to_datetime(daily_counts["date"])

    # S'assurer que tous les jours sont présents
    all_dates = pd.DataFrame({'date': date_range})
    daily_counts = pd.merge(all_dates, daily_counts, on='date', how='left').fillna(0)

    # Ajout du jour de la semaine
    daily_counts["weekday"] = daily_counts["date"].dt.day_name()
    daily_counts["is_weekend"] = daily_counts["date"].dt.dayofweek >= 5
    daily_counts["date_str"] = daily_counts["date"].dt.strftime('%d-%m (%a)')

    # Création du graphique avec Plotly
    fig = px.bar(
        daily_counts,
        x="date",
        y="trips",
        color="is_weekend",
        color_discrete_map={True: "orange", False: "royalblue"},
        labels={"date": "Date", "trips": "Nombre de trajets", "is_weekend": "Weekend"},
        title="Nombre de trajets par jour du mois"
    )

    # Mise en forme du graphique
    fig.update_layout(
        xaxis=dict(
            title=" ",
            tickmode='array',
            tickvals=daily_counts["date"],
            ticktext=daily_counts["date_str"],
            tickangle=45
        ),
        yaxis=dict(
            title=" "
        ),
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        height=600,
        width=1200,
        showlegend=False
    )

    fig.show()

    # Identification des jours significatifs
    # Filtrer pour éliminer les valeurs aberrantes (jours avec très peu de trajets)
    valid_days = daily_counts[daily_counts["trips"] > daily_counts["trips"].max() * 0.1]

    if not valid_days.empty:
        max_day = valid_days.loc[valid_days["trips"].idxmax()]
        min_day = valid_days.loc[valid_days["trips"].idxmin()]
        max_date = max_day["date"].strftime('%d/%m/%Y (%A)')
        min_date = min_day["date"].strftime('%d/%m/%Y (%A)')

        # Construction du message d'observation
        observation_msg = f"Le jour avec le plus grand nombre de courses est le {max_date} "
        observation_msg += f"avec {int(max_day['trips']):,} trajets. "
        observation_msg += f"Le jour avec le moins de courses est le {min_date} "
        observation_msg += f"avec {int(min_day['trips']):,} trajets. "

        # Analyse de la tendance
        valid_days = valid_days.reset_index(drop=True)
        x = np.arange(len(valid_days))
        y = valid_days["trips"].values
        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        if abs(r_value) > 0.5:
            trend = "forte"
        elif abs(r_value) > 0.3:
            trend = "modérée"
        else:
            trend = "faible"

        if slope > 0:
            direction = "hausse"
        else:
            direction = "baisse"

        # Ajout de l'analyse de tendance
        observation_msg += f"Sur l'ensemble du mois, on observe une {trend} tendance à la {direction} "
        observation_msg += f"de l'activité (r²={r_value**2:.2f})."

        # Ajout d'une interprétation possible
        if slope > 0 and r_value**2 > 0.3:
            observation_msg += " Cette hausse progressive pourrait indiquer une augmentation de la demande liée à des facteurs saisonniers ou événementiels."
        elif slope < 0 and r_value**2 > 0.3:
            observation_msg += " Cette baisse pourrait être liée à des facteurs saisonniers, des conditions météorologiques, ou à une augmentation de l'utilisation d'autres modes de transport."
    else:
        print("⚠️ Données insuffisantes pour analyser les tendances journalières")
        observation_msg = "Données insuffisantes pour analyser les tendances journalières."
else:
    print("⚠️ Données insuffisantes pour générer le graphique des trajets par jour")
    observation_msg = "Données insuffisantes pour analyser la répartition des trajets par jour."

In [ ]:
#| output: asis

# Affichage du bloc d'observation formaté
if 'observation_msg' in globals():
    print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
    print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
    print(f'  <p>{observation_msg}</p>')
    print('</div>')

# 💰 Chiffre d'Affaires et Revenus


In [ ]:
#| fig-cap: " "

# Série temporelle journalière
df['date'] = pd.to_datetime(df['pickup_datetime']).dt.date
daily_rev = df.groupby('date')['total_amount'].sum().reset_index(name='daily_revenue')
avg_rev = df['total_amount'].mean()

import plotly.express as px
fig = px.line(daily_rev, x='date', y='daily_revenue',
              labels={'date':'Date','daily_revenue':'Revenu total ($)'},
              title=f"Revenu total journalier et revenu moyen par course")
fig.update_layout(width=1200, height=600, plot_bgcolor='white')
fig.show()

In [ ]:
#| output: asis
max_rev = daily_rev['daily_revenue'].max()
min_rev = daily_rev['daily_revenue'].min()

observation_msg = f"Le revenu journalier maximal a atteint **${max_rev:,.0f}** et le minimum **${min_rev:,.0f}**. "
observation_msg += "Le revenu moyen par course s’établit à "
observation_msg += f"**${avg_rev:.2f}**, avec des variations marquées selon les jours."

print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
print(f'  <p>{observation_msg}</p>')
print('</div>')

# 🏷️ Répartition par Mode de Paiement

In [ ]:
payment_counts = df['payment_method'].value_counts(normalize=False).reset_index()
payment_counts.columns = ['mode','count']
payment_counts['pct'] = payment_counts['count'] / payment_counts['count'].sum() * 100

fig = px.bar(payment_counts, y='mode', x='count', orientation='h',
             text=payment_counts['pct'].map('{:.1f}%'.format),
             title="Nombre de courses par mode de paiement")
fig.update_layout(
    width=1200,
    height=600,
    plot_bgcolor='white',
    yaxis={'categoryorder':'total ascending'}
)
fig.show()

In [ ]:
#| output: asis
for _, row in payment_counts.iterrows():
    print(f"- {row['mode']}: {int(row['count']):,} courses ({row['pct']:.1f}%)")

# 🗺️ Carte des Prises en Charge par Zone


In [ ]:
# Charge le GeoJSON sous forme de GeoDataFrame
zones = gpd.read_file('data/taxi_zones.json')[['LocationID','zone','geometry']]

# Jointure avec les trajets
merged = zones.merge(
    df.groupby('PULocationID').size().reset_index(name='trips'),
    left_on='LocationID',
    right_on='PULocationID',
    how='left'
).fillna(0)

# Création de la carte choroplèthe sur fond blanc
fig = px.choropleth(
    merged,
    geojson=merged.geometry.__geo_interface__,  # passage du GeoDataFrame en GeoJSON inline
    locations=merged.index,
    color='trips',
    color_continuous_scale="Blues",
    range_color=(0, merged['trips'].max()),
    title="Nombre de trajets par zone de prise en charge"
)

fig.update_geos(
    fitbounds="locations",
    visible=False
)

fig.update_layout(
    width=1000,
    height=800,
    paper_bgcolor="white",
    plot_bgcolor="white",
    coloraxis_colorbar=dict(
        title="Trajets",
        tickformat=".0f"
    )
)

fig.show()

In [ ]:
#| output: asis
top_zone = merged.loc[merged['trips'].idxmax()]

observation_msg = f"La zone ayant enregistré le plus de prises en charge est **{top_zone['zone']}** "
observation_msg += f"avec **{int(top_zone['trips']):,}** trajets."

print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
print(f'  <p>{observation_msg}</p>')
print('</div>')

# 📏 Tarif Moyen par Mile

In [ ]:
df['fare_per_mile'] = df['total_amount'] / df['trip_distance']
# Box‑plot par tranche horaire
df['hour'] = pd.to_datetime(df['pickup_datetime']).dt.hour

fig = px.box(df, x='hour', y='fare_per_mile',
             labels={'hour':'Heure','fare_per_mile':'$ / mile'},
             title="Distribution du tarif ($) par mile selon l’heure")
fig.update_layout(width=1200, height=600, plot_bgcolor='white')
fig.show()

In [ ]:
#| output: asis
max_fpm = df['fare_per_mile'].max()
mean_fpm = df['fare_per_mile'].mean()
observation_msg = f"Le tarif moyen par mile est de **${mean_fpm:.2f}**, avec un maximum observé à **${max_fpm:.2f}**."
observation_msg += " Les valeurs extrêmes peuvent être dues à des trajets très courts avec minimum de prise en charge élevé."

print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
print(f'  <p>{observation_msg}</p>')
print('</div>')

# 👥 Nombre de Passagers et Indicateurs


In [ ]:
# Répartition passager_count
pc = df['passenger_count'].value_counts().reset_index()
pc.columns = ['passengers','count']

fig = px.bar(pc, x='passengers', y='count',
             labels={'passengers':' ','count':' '},
             title="Distribution du nombre de passagers par course")
fig.update_layout(width=1200, height=600, plot_bgcolor='white')
fig.show()

In [ ]:
# KPI par nombre de passagers
group = df.groupby('passenger_count').agg({
    'total_amount':'mean',
    'tip_amount':'mean'
}).reset_index().round(2)

# Création du graphique
fig = go.Figure()

# Barre revenu moyen
fig.add_trace(go.Bar(
    y=group['passenger_count'],
    x=group['total_amount'],
    name='Revenu moyen ($)',
    orientation='h',
    marker_color='royalblue'
))

# Barre pourboire moyen
fig.add_trace(go.Bar(
    y=group['passenger_count'],
    x=group['tip_amount'],
    name='Pourboire moyen ($)',
    orientation='h',
    marker_color='orange'
))

# Mise en forme du graphique
fig.update_layout(
    barmode='group',
    title="Revenu et pourboire moyen par nombre de passagers",
    xaxis_title="Montant moyen ($)",
    yaxis_title="Nombre de passagers",
    height=600,
    width=1000,
    paper_bgcolor="white",
    plot_bgcolor="white"
)

fig.show()

In [ ]:
#| output: asis
top_group = group.loc[group['total_amount'].idxmax()]

observation_msg = f"Les trajets avec **{int(top_group['passenger_count'])} passager(s)** rapportent en moyenne le revenu le plus élevé à **${top_group['total_amount']:.2f}**."
observation_msg += f" Le pourboire moyen dans cette catégorie est de **${top_group['tip_amount']:.2f}**."

print('<div style="background-color: #f0f7fb; border-left: 5px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 3px;">')
print('  <h4 style="margin-top: 0; color: #3498db;">Observation</h4>')
print(f'  <p>{observation_msg}</p>')
print('</div>')

# 📌 Synthèse du Mois


In [ ]:
#| echo: false
# Variables pour la synthèse
if not df.empty and "trip_distance" in df.columns and "hour" in df.columns:
    # Top 3 des heures les plus actives
    hourly_trips = df.groupby("hour").size()
    peak_hours = hourly_trips.sort_values(ascending=False).head(3).index.tolist()
    peak_hours_range = f"{min(peak_hours)}h et {max(peak_hours)}h"

    avg_distance_rounded = round(avg_distance, 2)
else:
    peak_hours_range = "17h et 20h"
    avg_distance_rounded = 0

In [ ]:
#| output: asis
print(f"- 📈 L’activité maximale se concentre généralement entre **{peak_hours_range}** selon les données disponibles.")
print(f"- 🛣️ Distance moyenne des trajets : **{avg_distance_rounded:.2f} miles**")
print(f"- 💵 Revenu moyen par course : **${avg_rev:.2f}**")
print(f"- 📊 Zone la plus fréquentée : **{top_zone['zone']}** avec **{int(top_zone['trips']):,}** prises en charge.")
print(f"- 🏷️ Mode de paiement le plus utilisé : **{top_mode['mode']}** ({top_mode['pct']:.1f}%)")
print(f"- 📌 Tarif moyen par mile : **${mean_fpm:.2f}**")

# 🚀 Recommandations pour les Prochains Mois

* Pousser l’analyse du chiffre d’affaires journalier et des variations hebdomadaires.

* Suivre l’évolution des modes de paiement (essor ou déclin de certaines pratiques).

* Localiser les zones les plus actives pour adapter les ressources taxi et campagnes marketing.

* Suivre les flux inter-zones principaux pour planifier la logistique et anticiper les congestions.

* Intégrer des analyses de tarifs par mile et durées pour détecter les anomalies tarifaires.

* Étendre le rapport aux indicateurs de pourboire et passagers pour affiner les segments de clientèle.
